In [1]:
!pip install torch torchvision scikit-learn seaborn tqdm


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!git clone https://github.com/pratikkayal/PlantDoc-Dataset.git

Cloning into 'PlantDoc-Dataset'...
remote: Enumerating objects: 2670, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 2670 (delta 22), reused 22 (delta 22), pack-reused 2635 (from 1)
Receiving objects: 100% (2670/2670), 932.92 MiB | 40.34 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Updating files: 100% (2581/2581), done.


In [3]:
import os

train_dir = "./PlantDoc-Dataset/train"
test_dir = "./PlantDoc-Dataset/test"

# Gather the folder names (only subdirectories) in each directory
train_folders = [d for d in os.listdir(train_dir)
                 if os.path.isdir(os.path.join(train_dir, d))]
test_folders = [d for d in os.listdir(test_dir)
                if os.path.isdir(os.path.join(test_dir, d))]

# Find folders that exist in train but not in test
extra_folders_in_train = set(train_folders) - set(test_folders)

# Print results
if extra_folders_in_train:
    print("Folders in train but not in test:")
    for folder in extra_folders_in_train:
        print(folder)
else:
    print("No extra folders in train that are missing from test!")


Folders in train but not in test:
Tomato two spotted spider mites leaf


In [4]:
#remove the folder 'Tomato two spotted spider mites leaf' which is in train, but not in test
import os
import shutil

train_dir = "./PlantDoc-Dataset/train"
test_dir  = "./PlantDoc-Dataset/test"

# Find folders in train but not in test
train_folders = set(os.listdir(train_dir))
test_folders  = set(os.listdir(test_dir))
extra_folders = train_folders - test_folders

for folder in extra_folders:
    folder_path = os.path.join(train_dir, folder)
    if os.path.isdir(folder_path):
        print("Removing extra folder from train:", folder_path)
        shutil.rmtree(folder_path)  # Delete the entire folder

# Re-initialize the ImageFolder datasets now that you have identical classes
from torchvision import datasets

train_dataset = datasets.ImageFolder(root=train_dir)  # ...
test_dataset  = datasets.ImageFolder(root=test_dir)   # ...

print("Train classes:", train_dataset.classes)
print("Test classes :", test_dataset.classes)


Removing extra folder from train: ./PlantDoc-Dataset/train/Tomato two spotted spider mites leaf
Train classes: ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'grape leaf', 'grape leaf black rot']
Test classes : ['Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf', 'Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabe

In [5]:
# Count the number of image files
# 27 classes and 2,576 images.
import os

# Define the image extensions you want to count
extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')

# Function to count images recursively in a given folder
def count_images(folder):
    return sum(
        1 for root, dirs, files in os.walk(folder)
        for file in files if file.lower().endswith(extensions)
    )

# Paths to your train and test folders
test_folder = './PlantDoc-Dataset/test'
train_folder = './PlantDoc-Dataset/train'

# Count images in each folder
image_count_test = count_images(test_folder)
image_count_train = count_images(train_folder)

print(f"Number of images in the 'test' folder: {image_count_test}")
print(f"Number of images in the 'train' folder: {image_count_train}")
print(f"Total Number of images: {image_count_test + image_count_train}")



Number of images in the 'test' folder: 236
Number of images in the 'train' folder: 2340
Total Number of images: 2576


In [6]:
#sample image size
from PIL import Image

image_path = './PlantDoc-Dataset/test/Apple leaf/apple-leaf-14319997.jpg'
img = Image.open(image_path)
width, height = img.size

print(f"Width: {width}, Height: {height}")


Width: 1300, Height: 1174


In [7]:
#generate plantDoc unlabel images

import os
import shutil
import random
from tqdm import tqdm

# Define source directories and target folder
train_dir = './PlantDoc-Dataset/train'
test_dir = './PlantDoc-Dataset/test'
unlabel_dir = './PlantDoc-Dataset/unlabel'  # Target folder for all images

# Create the unlabel folder if it doesn't exist
os.makedirs(unlabel_dir, exist_ok=True)

# Function to collect image paths from subfolders
def collect_images(source_dir):
    image_paths = []
    for label in os.listdir(source_dir):  # Loop through each subfolder (label)
        label_path = os.path.join(source_dir, label)
        if os.path.isdir(label_path):
            for img in os.listdir(label_path):
                img_path = os.path.join(label_path, img)
                if img.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Case-sensitive check
                    image_paths.append(img_path)
    return image_paths

# Collect all image paths from train and test directories
print("Collecting images from train_dir...")
train_images = collect_images(train_dir)
print("Collecting images from test_dir...")
test_images = collect_images(test_dir)

# Combine both sets and shuffle them randomly
all_images = train_images + test_images
random.shuffle(all_images)  # Shuffle the images randomly

# Function to copy images to the unlabel folder
def copy_images_to_unlabel(image_paths, target_dir):
    for img_path in tqdm(image_paths, desc="Copying and Shuffling Images"):
        filename = os.path.basename(img_path)  # Keep the original filename
        target_path = os.path.join(target_dir, filename)

        # Avoid overwriting files with the same name
        counter = 1
        while os.path.exists(target_path):
            name, ext = os.path.splitext(filename)
            target_path = os.path.join(target_dir, f"{name}_{counter}{ext}")
            counter += 1

        shutil.copy(img_path, target_path)  # Copy the image to the unlabel folder

# Copy and shuffle images
copy_images_to_unlabel(all_images, unlabel_dir)

print(f"All {len(all_images)} images have been copied and randomly placed in {unlabel_dir}!")


Copying and Shuffling Images: 100%|██████████| 2576/2576 [00:00<00:00, 2762.43it/s]

All 2576 images have been copied and randomly placed in ./PlantDoc-Dataset/unlabel!


In [8]:
#verify the total number of plantDoc unlabel images
import os

# Define the path to the unlabel folder
unlabel_dir = './PlantDoc-Dataset/unlabel'

# Get a list of all image files in the directory
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')  # Case-sensitive check
image_files = [f for f in os.listdir(unlabel_dir) if f.lower().endswith(image_extensions)]

# Print the total number of images
print(f"Total number of images in '{unlabel_dir}': {len(image_files)}")


Total number of images in './PlantDoc-Dataset/unlabel': 2576


In [9]:
!pip install kornia    #install GPU augmentation library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!pip install byol-pytorch


[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git   #download plantVillage dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163235, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 163235 (delta 2), reused 0 (delta 0), pack-reused 163229 (from 1)
Receiving objects: 100% (163235/163235), 2.00 GiB | 44.82 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (182401/182401), done.
